DSC140A SuperHW

The problem. Consider the words “meilleur” and “mejor”. In English, both of these words mean “best”
– one in French and the other in Spanish. Even if you don’t know how to speak either of these languages,
you might be able to guess which word is Spanish and which is French based on the spelling. For example,
the word “meilleur” looks more like a French word than a Spanish word due to it containing “ei” and ending
in “eur”. On the other hand, the word “mejor” looks more like a Spanish word than a French word due to it
containing “j” and ending in “or”. This suggests that there is some statistical structure in the words of these
languages that we can use to build a machine learning model capable of distinguishing between French and
Spanish without actually understanding the words themselves.

Your goal in this problem is to build a machine learning model that can take a word as input and predict
whether it is Spanish or French.

• train_words: a list of n strings, each one of them a word (in either Spanish or French)


• train_labels: a list of n strings, each one of them either "spanish" or "french", indicating the
language of the corresponding word in train_words

•  test_words: a list of m strings, each one of them a word (in either Spanish or French)


This function should return a list of m strings, each one of them either "spanish" or "french", indicating
your classifier’s prediction for the language of the corresponding word in test_words.
Your classify() function is responsible for training your machine learning model on the training data and
then using that model to make predictions on the test data.


A good choice of features is important. You might want to consider using the frequency of different
letters or pairs of letters in each word as features. For example, one of your features might be whether
“el” appears in the word. You do not need to create all of these features by hand – you can use Python
to help you generate them.

Don’t confuse training accuracy with test accuracy. It is possible to achieve 90%+ training accuracy
on this data set, but that doesn’t mean your model will generalize well to the test set.
2
• Be careful to avoid overfitting! If you use too many features or too complex of a model, you may find
that your model performs well on the training data but poorly on the test data.
• Start with the simplest models first. We have learned some models in this class that can be implemented
using only a couple lines of code (with numpy).

In [13]:
import numpy as np
from collections import Counter

def classify(train_words, train_labels, test_words):
    """
    Classify words as either Spanish or French
    
    Parameters:
    - train_words: List of strings (words) for training
    - train_labels: List of strings (either "spanish" or "french") matching train_words
    - test_words: List of strings (words) to classify
    
    Returns:
    - List of predicted labels ("spanish" or "french") for test_words
    """
    # Implement a Naive Bayes classifier with character n-gram features
    
    # Split training data by class
    spanish_words = [word.lower() for word, label in zip(train_words, train_labels) if label == "spanish"]
    french_words = [word.lower() for word, label in zip(train_words, train_labels) if label == "french"]
    
    # Get prior probabilities
    total_words = len(train_words)
    p_spanish = len(spanish_words) / total_words
    p_french = len(french_words) / total_words
    
    # Extract features: single characters, bigrams, and ending patterns
    spanish_chars = extract_features(spanish_words)
    french_chars = extract_features(french_words)
    
    # Calculate the sum of feature counts for each class
    spanish_total = sum(spanish_chars.values())
    french_total = sum(french_chars.values())
    
    # Get all unique features
    all_features = set(list(spanish_chars.keys()) + list(french_chars.keys()))
    feature_count = len(all_features)
    
    # Make predictions for each test word
    predictions = []
    for word in test_words:
        word = word.lower()
        
        # Calculate log probabilities for each class (using log to avoid underflow)
        log_p_spanish = np.log(p_spanish)
        log_p_french = np.log(p_french)
        
        # Extract features from the test word
        word_features = get_word_features(word)
        
        # Update probabilities based on features
        for feature in word_features:
            # Handle Spanish probability (with Laplace smoothing)
            if feature in spanish_chars:
                log_p_spanish += np.log((spanish_chars[feature] + 1) / (spanish_total + feature_count))
            else:
                log_p_spanish += np.log(1 / (spanish_total + feature_count))
            
            # Handle French probability (with Laplace smoothing)
            if feature in french_chars:
                log_p_french += np.log((french_chars[feature] + 1) / (french_total + feature_count))
            else:
                log_p_french += np.log(1 / (french_total + feature_count))
        
        # Add slight bias toward French to fix our imbalance from analysis
        log_p_french += 0.05
        
        # Make prediction based on higher probability
        if log_p_spanish > log_p_french:
            predictions.append("spanish")
        else:
            predictions.append("french")
    
    return predictions

def extract_features(words):
    """
    Extract character-level features from a list of words
    """
    features = Counter()
    
    for word in words:
        # Add single characters
        for char in word:
            features[f"CHAR_{char}"] += 1
        
        # Add bigrams
        for i in range(len(word) - 1):
            bigram = word[i:i+2]
            features[f"BI_{bigram}"] += 1
        
        # Add trigrams (very distinctive for language identification)
        for i in range(len(word) - 2):
            trigram = word[i:i+3]
            features[f"TRI_{trigram}"] += 1
        
        # Add special features for word endings (very important for language identification)
        for length in [1, 2, 3]:
            if len(word) >= length:
                ending = word[-length:]
                features[f"END_{ending}"] += 3  # Give more weight to endings
        
        # Add special features for word beginnings
        for length in [1, 2, 3]:
            if len(word) >= length:
                beginning = word[:length]
                features[f"START_{beginning}"] += 2  # Give more weight to beginnings
                
        # Add word length as a feature
        length_range = min(len(word) // 2, 5)  # Group lengths to avoid overfitting
        features[f"LEN_{length_range}"] += 1
        
        # Add common French patterns with higher weight
        french_patterns = ["eu", "ou", "ai", "ei", "au", "eau", "oi", "ie", "tion", "eux", "aux"]
        for pattern in french_patterns:
            if pattern in word:
                features[f"FR_PATTERN_{pattern}"] += 3
        
        # Add common Spanish patterns with higher weight
        spanish_patterns = ["os", "ar", "er", "ir", "mente", "dad", "cion", "ll", "rr", "ia", "io"]
        for pattern in spanish_patterns:
            if pattern in word:
                features[f"ES_PATTERN_{pattern}"] += 3
    
    return features

def get_word_features(word):
    """
    Extract features from a single word
    """
    features = []
    
    # Add single characters
    for char in word:
        features.append(f"CHAR_{char}")
    
    # Add bigrams
    for i in range(len(word) - 1):
        bigram = word[i:i+2]
        features.append(f"BI_{bigram}")
    
    # Add special features for word endings
    for length in [1, 2, 3]:
        if len(word) >= length:
            ending = word[-length:]
            features.append(f"END_{ending}")
    
    # Add special features for word beginnings
    for length in [1, 2, 3]:
        if len(word) >= length:
            beginning = word[:length]
            features.append(f"START_{beginning}")
            
    # Add word length as a feature
    length_range = min(len(word) // 2, 5)
    features.append(f"LEN_{length_range}")
    
    return features

In [16]:
import numpy as np
from sklearn.model_selection import KFold

def evaluate_classifier(all_words, all_labels, k=5):
    """Perform k-fold cross-validation to estimate model accuracy"""
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    accuracies = []
    
    for train_idx, test_idx in kf.split(all_words):
        train_words = [all_words[i] for i in train_idx]
        train_labels = [all_labels[i] for i in train_idx]
        test_words = [all_words[i] for i in test_idx]
        test_labels = [all_labels[i] for i in test_idx]
        
        predictions = classify(train_words, train_labels, test_words)
        accuracy = sum(p == t for p, t in zip(predictions, test_labels)) / len(test_labels)
        accuracies.append(accuracy)
    
    return np.mean(accuracies), np.std(accuracies)

# Usage
mean_acc, std_acc = evaluate_classifier(train_words, train_labels)
print(f"Estimated accuracy: {mean_acc:.4f} ± {std_acc:.4f}")

Estimated accuracy: 0.8469 ± 0.0473


In [17]:
def analyze_feature_importance(train_words, train_labels):
    """Identify which features best distinguish between languages"""
    spanish_words = [w for w, l in zip(train_words, train_labels) if l == "spanish"]
    french_words = [w for w, l in zip(train_words, train_labels) if l == "french"]
    
    spanish_features = extract_features(spanish_words)
    french_features = extract_features(french_words)
    
    # Normalize counts to frequencies
    total_spanish = sum(spanish_features.values())
    total_french = sum(french_features.values())
    
    important_features = []
    for feature in set(list(spanish_features.keys()) + list(french_features.keys())):
        sp_freq = spanish_features.get(feature, 0) / total_spanish
        fr_freq = french_features.get(feature, 0) / total_french
        importance = abs(sp_freq - fr_freq)
        important_features.append((feature, importance, sp_freq, fr_freq))
    
    return sorted(important_features, key=lambda x: x[1], reverse=True)[:20]

In [18]:
def confusion_matrix(train_words, train_labels, test_words, test_labels):
    """Generate confusion matrix to identify error patterns"""
    predictions = classify(train_words, train_labels, test_words)
    
    # Initialize counts
    tp = fp = tn = fn = 0
    for pred, true in zip(predictions, test_labels):
        if pred == "spanish" and true == "spanish":
            tp += 1
        elif pred == "spanish" and true == "french":
            fp += 1
        elif pred == "french" and true == "french":
            tn += 1
        elif pred == "french" and true == "spanish":
            fn += 1
    
    # Calculate metrics
    accuracy = (tp + tn) / len(test_labels)
    precision_spanish = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall_spanish = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return {
        "accuracy": accuracy,
        "precision_spanish": precision_spanish,
        "recall_spanish": recall_spanish,
        "confusion_matrix": [[tp, fn], [fp, tn]]
    }

In [19]:
def test_edge_cases(train_words, train_labels):
    """Test classifier on specially crafted challenging cases"""
    edge_cases = [
        # Words that could be either language
        "animal", "central", "radio", "normal",
        
        # Very short words
        "no", "si", "la", "le",
        
        # Words with language-specific patterns
        "español", "français", "biblioteca", "bibliothèque",
    ]
    
    predictions = classify(train_words, train_labels, edge_cases)
    for word, pred in zip(edge_cases, predictions):
        print(f"'{word}' classified as {pred}")